In [151]:
import sklearn
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/prateek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/prateek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [152]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score
import pickle

In [153]:
url='https://github.com/prateekmittal154/Movie-Review-Sentiment-Analysis/blob/main/IMDB-Dataset.csv?raw=true'

In [154]:
dataset= pd.read_csv('IMDB-Dataset.csv')
print(dataset.shape)
dataset.head()

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [155]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [156]:
dataset.sentiment.value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

In [157]:
dataset.sentiment.replace('positive', 1, inplace=True)
dataset.sentiment.replace('negative', 0, inplace=True)

In [158]:
def to_lower(text):
    return text.lower()

dataset.review = dataset.review.apply(to_lower)
dataset.review[0]


"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.<br /><br />the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.<br /><br />it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />i would say the main appeal of the show is due to the fa

In [159]:
def clean(text):
    cleaned = re.compile(r'<.*?>')
    return re.sub(cleaned,'',text)

dataset.review = dataset.review.apply(clean)
dataset.review[0]

"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.i would say the main appeal of the show is due to the fact that it goes where other shows wo

In [160]:
def is_special(text):
    rem = ''
    for i in text:
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '
    return rem

dataset.review = dataset.review.apply(is_special)
dataset.review[0]


'one of the other reviewers has mentioned that after watching just 1 oz episode you ll be hooked  they are right  as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence  which set in right from the word go  trust me  this is not a show for the faint hearted or timid  this show pulls no punches with regards to drugs  sex or violence  its is hardcore  in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary  it focuses mainly on emerald city  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  em city is home to many  aryans  muslims  gangstas  latinos  christians  italians  irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wo

In [161]:
def rem_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    return [w for w in words if w not in stop_words]

data.review = data.review.apply(rem_stopwords)
data.review[0]

['pete',
 'meteor',
 'seen',
 'referred',
 'authentic',
 'gem',
 'caught',
 'movie',
 'saturday',
 'afternoon',
 'kinda',
 'wished',
 'apparently',
 'noted',
 'mike',
 'myers',
 'first',
 'dramatic',
 'role',
 'unfortunately',
 'movie',
 'first',
 'thing',
 'notice',
 'pete',
 'meteor',
 'suffers',
 'complete',
 'lack',
 'plot',
 'rather',
 'bunch',
 'little',
 'subplots',
 'really',
 'connected',
 'go',
 'anywhere',
 'young',
 'orphan',
 'mickey',
 'lives',
 'younger',
 'brother',
 'sister',
 'grandmother',
 'meteorite',
 'lands',
 'garden',
 'eventually',
 'collected',
 'scientist',
 'named',
 'hugh',
 'despite',
 'title',
 'movie',
 'mickey',
 'one',
 'lays',
 'claim',
 'meteor',
 'storyline',
 'actually',
 'little',
 'meteor',
 'shape',
 'form',
 'several',
 'characters',
 'subplots',
 'introduced',
 'like',
 'said',
 'really',
 'go',
 'anywhere',
 'things',
 'go',
 'cute',
 'family',
 'movie',
 'dark',
 'thriller',
 'pretty',
 'fast',
 'pete',
 'mike',
 'myers',
 'introduced',
 's

In [162]:
def stem_txt(text):
    ss = SnowballStemmer('english')
    return " ".join([ss.stem(w) for w in text])

data.review = data.review.apply(stem_txt)
data.review[0]

'pete meteor seen refer authent gem caught movi saturday afternoon kinda wish appar note mike myer first dramat role unfortun movi first thing notic pete meteor suffer complet lack plot rather bunch littl subplot realli connect go anywher young orphan mickey live younger brother sister grandmoth meteorit land garden eventu collect scientist name hugh despit titl movi mickey one lay claim meteor storylin actual littl meteor shape form sever charact subplot introduc like said realli go anywher thing go cute famili movi dark thriller pretti fast pete mike myer introduc start play role micki blame death parent though never realli told happen hugh romanc problem girlfriend get past cosmic fascin pete drug relat past start catch temporarili draw girlfriend rest cast mix shadi charact start tri kill would seem main plot realiz even role halfway film make everyth els feel like pointless filler hugh girl problem mickey bad behavior meteor earth much plot disjoint half pointless good cast charac

In [163]:
data.head()

,review,sentiment
0,pete meteor seen refer authent gem caught movi...,0
1,cast realli help make pleasant surpris cut nor...,1
2,download movi yesterday internet site qualiti ...,0
3,point film becom bad good compel aw darkhunt s...,0
4,rate tv 14 sexual content languag rememb hear ...,1


In [164]:
# sample 5% of data to be used as unseen data

data = dataset.sample(frac=0.95, random_state=786)
data_unseen = dataset.drop(data.index)
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)


# print the revised shape
print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))


Data for Modeling: (47500, 2)
Unseen Data For Predictions: (2500, 2)


In [165]:
X = np.array(data.iloc[:,0].values)
y = np.array(data.sentiment.values)
cv = CountVectorizer(max_features=1000)
X = cv.fit_transform(data.review).toarray()
print("X.shape = ",X.shape)
print("y.shape = ",y.shape)

X.shape =  (47500, 1000)
y.shape =  (47500,)


In [166]:
trainx = X
trainy = y

print("Train shapes : trainx = {}, trainy = {}".format(trainx.shape,trainy.shape))

Train shapes : trainx = (47500, 1000), trainy = (47500,)


In [167]:
X1= np.array(data_unseen.iloc[:,0].values)
y1 = np.array(data_unseen.sentiment.values)
cv= CountVectorizer(max_features=1000)
X1=cv.fit_transform(data_unseen.review).toarray()
print("X1.shape= ", X1.shape)
print("y1.shape= ", y1.shape)

X1.shape=  (2500, 1000)
y1.shape=  (2500,)


In [168]:
testx = X1
testy = y1

print("Test shapes : testx = {}, testy = {}".format(testx.shape,testy.shape))

Test shapes : testx = (2500, 1000), testy = (2500,)


In [169]:
gnb = GaussianNB()
gnb.fit(trainx, trainy)

GaussianNB()

In [170]:
ypg = gnb.predict(testx)


print("Gaussian = ",accuracy_score(testy,ypg))


Gaussian =  0.5636


In [183]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(testy,ypg)

print(cm)

[[696 558]
 [533 713]]


In [172]:
mnb= MultinomialNB(alpha=1.0,fit_prior=True)
mnb.fit(trainx, trainy)

ypm= mnb.predict(testx)

print("Multinomial = ", accuracy_score(testy, ypm))

Multinomial =  0.592


In [186]:
from sklearn.metrics import confusion_matrix

cm1 = confusion_matrix(testy,ypm)

print(cm1)

[[926 328]
 [692 554]]


In [174]:
bnb= BernoulliNB(alpha=1.0,fit_prior=True)
bnb.fit(trainx, trainy)

BernoulliNB()

In [177]:
ypb= bnb.predict(testx)

print("Bernoulli = ", accuracy_score(testy, ypb))

Bernoulli =  0.6108


In [184]:
from sklearn.metrics import confusion_matrix

cm2 = confusion_matrix(testy,ypb)

print(cm2)

[[625 629]
 [344 902]]


In [187]:
pickle.dump(bnb,open('model1.pkl','wb'))

In [188]:
rev =  """Terrible. Complete trash. Brainless tripe. Insulting to anyone who isn't an 8 year old fan boy. Im actually pretty disgusted that this movie is making the money it is - what does it say about the people who brainlessly hand over the hard earned cash to be 'entertained' in this fashion and then come here to leave a positive 8.8 review?? Oh yes, they are morons. Its the only sensible conclusion to draw. How anyone can rate this movie amongst the pantheon of great titles is beyond me.

So trying to find something constructive to say about this title is hard...I enjoyed Iron Man? Tony Stark is an inspirational character in his own movies but here he is a pale shadow of that...About the only 'hook' this movie had into me was wondering when and if Iron Man would knock Captain America out...Oh how I wished he had :( What were these other characters anyways? Useless, bickering idiots who really couldn't organise happy times in a brewery. The film was a chaotic mish mash of action elements and failed 'set pieces'...

I found the villain to be quite amusing.

And now I give up. This movie is not robbing any more of my time but I felt I ought to contribute to restoring the obvious fake rating and reviews this movie has been getting on IMDb."""
f1 = clean(rev)
f2 = is_special(f1)
f3 = to_lower(f2)
f4 = rem_stopwords(f3)
f5 = stem_txt(f4)

bow,words = [],word_tokenize(f5)
for word in words:
    bow.append(words.count(word))
#np.array(bow).reshape(1,3000)
#bow.shape
word_dict = cv.vocabulary_
pickle.dump(word_dict,open('bow.pkl','wb'))

In [190]:
inp = []
for i in word_dict:
    inp.append(f5.count(i[0]))
y_pred = bnb.predict(np.array(inp).reshape(1,1000))

In [191]:
print(y_pred)

[0]
